<a href="https://colab.research.google.com/github/PakwhanNK/DP-newsletter-content-analysis/blob/main/01_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newsletter Data Collection
This notebook collects data from BigQuery. The information from Mailchimp is connected to Bigquery using connectors.

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv

In [1]:
from google.colab import auth
auth.authenticate_user()

In [6]:
from google.cloud import bigquery

In [7]:
from google.colab import userdata
PROJECT_ID = userdata.get('PROJECT_ID')
DATASET_ID = userdata.get('DATASET_ID')
client = bigquery.Client(project=PROJECT_ID)

In [8]:
datasets = list(client.list_datasets())
print("Datasets:")
for ds in datasets:
    print(f"  • {ds.dataset_id}")

Datasets:
  • Querying
  • analytics_344298334
  • analytics_366994706
  • analytics_372418130
  • dpdatabigquery
  • dpdatabigquery_facebook_pages
  • dpdatabigquery_stg_facebook_pages
  • facebook_pages
  • facebook_pages_facebook_pages
  • facebook_pages_stg_facebook_pages
  • fivetran_covert_tuft_staging
  • fivetran_metadata
  • fivetran_salaries_penned_staging
  • instagram_business
  • instagram_business_instagram_business
  • instagram_business_stg_instagram_business
  • mailchimp
  • mailchimp_mailchimp
  • mailchimp_stg_mailchimp


In [9]:
# See what tables Fivetran created
tables = client.list_tables(DATASET_ID)

print(f"Tables in {DATASET_ID}:\n")
table_list = []

for table in tables:
    # Get table details
    table_ref = f"{PROJECT_ID}.{DATASET_ID}.{table.table_id}"
    table_obj = client.get_table(table_ref)

    table_info = {
        'table_name': table.table_id,
        'rows': f"{table_obj.num_rows:,}",
        'size_mb': f"{table_obj.num_bytes / 1024 / 1024:.2f}"
    }
    table_list.append(table_info)

    print(f"{table.table_id}")
    print(f"Rows: {table_obj.num_rows:,}")
    print(f"Size: {table_obj.num_bytes / 1024 / 1024:.2f} MB")
    print()

# Create summary DataFrame
df_tables = pd.DataFrame(table_list)
df_tables

Tables in mailchimp:

campaign
Rows: 2,970
Size: 1.69 MB

campaign_recipient
Rows: 18,098,537
Size: 1141.86 MB

campaign_recipient_activity
Rows: 15,499,445
Size: 1802.61 MB

campaign_variate_combination
Rows: 65
Size: 0.01 MB

interest
Rows: 14
Size: 0.00 MB

interest_category
Rows: 2
Size: 0.00 MB

interest_member
Rows: 48,381
Size: 2.54 MB

list
Rows: 2
Size: 0.00 MB

member
Rows: 42,782
Size: 10.22 MB

segment
Rows: 91
Size: 0.01 MB

segment_member
Rows: 65,261
Size: 3.75 MB

tag_member
Rows: 13,732
Size: 0.65 MB

tags
Rows: 71
Size: 0.00 MB



,table_name,rows,size_mb
0,campaign,"2,970",1.69
1,campaign_recipient,"18,098,537",1141.86
2,campaign_recipient_activity,"15,499,445",1802.61
3,campaign_variate_combination,65,0.01
4,interest,14,0.00
5,interest_category,2,0.00
6,interest_member,"48,381",2.54
7,list,2,0.00
8,member,"42,782",10.22
9,segment,91,0.01


In [16]:
table_ref = f"{PROJECT_ID}.{DATASET_ID}.campaign"
campaign = client.get_table(table_ref)

print(f"CAMPAIGN table has {len(campaign.schema)} columns:\n")

# Show all columns
for field in campaign.schema:
    print(f"  • {field.name:30s} ({field.field_type})")

CAMPAIGN table has 40 columns:

  • id                             (STRING)
  • type                           (STRING)
  • create_time                    (TIMESTAMP)
  • archive_url                    (STRING)
  • long_archive_url               (STRING)
  • status                         (STRING)
  • send_time                      (TIMESTAMP)
  • content_type                   (STRING)
  • list_id                        (STRING)
  • segment_text                   (STRING)
  • segment_id                     (STRING)
  • title                          (STRING)
  • to_name                        (STRING)
  • authenticate                   (BOOLEAN)
  • timewarp                       (BOOLEAN)
  • subject_line                   (STRING)
  • from_name                      (STRING)
  • reply_to                       (STRING)
  • use_conversation               (BOOLEAN)
  • folder_id                      (STRING)
  • auto_footer                    (BOOLEAN)
  • inline_css                    

In [21]:
query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.campaign`
"""

campaign = client.query(query).to_dataframe()
campaign.head()

,id,type,create_time,archive_url,long_archive_url,status,send_time,content_type,list_id,segment_text,...,track_opens,track_ecomm_360,google_analytics,_fivetran_deleted,winning_combination_id,winning_campaign_id,winner_criteria,wait_time,test_size,_fivetran_synced
0,9c4e1fbdfd,regular,2016-10-17 02:06:55+00:00,http://eepurl.com/ckn9ML,https://us2.campaign-archive.com/?u=a96885e3aa...,sent,2016-10-17 08:30:00+00:00,template,370b4800ba,,...,True,False,,False,None,None,None,<NA>,<NA>,2025-11-08 06:07:05.259000+00:00
1,2b029fa22f,regular,2017-01-29 01:48:08+00:00,http://eepurl.com/czx1pj,https://us2.campaign-archive.com/?u=a96885e3aa...,sent,2017-01-30 00:30:00+00:00,template,370b4800ba,,...,True,False,,False,None,None,None,<NA>,<NA>,2025-11-08 06:07:05.271000+00:00
2,b6e9a8af00,regular,2016-09-12 01:59:18+00:00,http://eepurl.com/cfi2gj,https://us2.campaign-archive.com/?u=a96885e3aa...,sent,2016-09-12 08:00:00+00:00,template,370b4800ba,,...,True,False,,False,None,None,None,<NA>,<NA>,2025-11-08 06:07:05.255000+00:00
3,0c4ca3e668,regular,2017-02-08 06:16:55+00:00,http://eepurl.com/cA4grn,https://us2.campaign-archive.com/?u=a96885e3aa...,sent,2017-02-08 09:30:00+00:00,template,370b4800ba,,...,True,False,,False,None,None,None,<NA>,<NA>,2025-11-08 06:07:05.276000+00:00
4,6121a383b6,regular,2017-02-14 03:12:30+00:00,http://eepurl.com/cBUVSr,https://us2.campaign-archive.com/?u=a96885e3aa...,sent,2017-02-14 09:00:00+00:00,template,370b4800ba,,...,True,False,,False,None,None,None,<NA>,<NA>,2025-11-08 06:07:05.278000+00:00


## Scraping data
Check archive URLs


In [25]:
# Do you have URLs to the actual newsletter content?
query = f"""
SELECT
    id,
    subject_line,
    send_time,
    archive_url,
    long_archive_url
FROM `{PROJECT_ID}.{DATASET_ID}.campaign`
WHERE
    status = 'sent'
    AND archive_url IS NOT NULL
ORDER BY send_time DESC
"""

df_urls = client.query(query).to_dataframe()

print(f"Found {len(df_urls)} campaigns with archive URLs")
print("\nSample URLs:")
df_urls[['subject_line', 'archive_url']].head()

Found 2841 campaigns with archive URLs

Sample URLs:


,subject_line,archive_url
0,Your Weekly Toast: On the Boarder–line,http://eepurl.com/jrnaSM
1,"Voices of Penn: Mamdani's victory, grade infla...",http://eepurl.com/jrolUM
2,🏈 Quaker Nation: 2025 Homecoming Preview,http://eepurl.com/jrn9FA
3,Friday Morning: Men’s basketball using NIL col...,http://eepurl.com/jrn_vI
4,🏀 Quaker Nation: 2025 Basketball Preview,http://eepurl.com/jrhwlw
